# To-Do: Vision API와 Gradio를 연동하여 웹 앱 만들기
이 노트북은 Azure Vision API를 사용하여 이미지 분석 기능을 함수화하고, 최종적으로 Gradio를 사용하여 사용자가 직접 이미지를 분석할 수 있는 웹 인터페이스를 만드는 과정을 안내합니다.
각 단계별 To-Do 주석을 따라 코드를 완성하고 실행해보세요.

In [1]:
# To-Do: 1. 필요한 라이브러리 가져오기
# requests: API 요청을 보내기 위한 라이브러리
# PIL (Pillow): 이미지 처리 (열기, 그리기 등)를 위한 라이브러리
# io.BytesIO: 이미지 데이터를 메모리 상에서 다루기 위한 라이브러리
# gradio: 웹 UI를 쉽게 만들기 위한 라이브러리
import requests
from PIL import Image, ImageDraw
from io import BytesIO
import gradio as gr

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# To-Do: 2. API 설정 및 기본 변수 정의
# IMAGE_URL: 분석할 기본 이미지 URL
# BASE_ENDPOINT: Azure Cognitive Services Vision API의 기본 엔드포인트 주소
# API_KEY: Vision API 사용을 위한 구독 키
IMAGE_URL = "https://cdn.pixabay.com/photo/2019/08/25/13/34/dogs-4429513_1280.jpg"
BASE_ENDPOINT = "https://fimtrus-vision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
API_KEY="1Xvo96LQT7LFAe6FgQUeO3f8UwOfELLcvZNJThBQW6LJsBdUhwzrJQQJ99BGACYeBjFXJ3w3AAAFACOGxEEM"

In [ ]:
# To-Do: 3. 유틸리티 함수 정의 (random_color, get_font)
# random_color(): 분석 결과를 시각화할 때 사용할 랜덤 색상을 생성합니다.
# get_font(): 운영체제에 맞는 한글 지원 폰트를 가져옵니다. (macOS, Windows, Linux 지원)
def random_color():
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def get_font():
    from PIL import ImageFont
    import platform
    font_size = 20
    try:
        if platform.system() == "Windows":
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:
            return ImageFont.load_default(size=font_size)
    except IOError:
        return ImageFont.load_default(size=font_size)

In [ ]:
# To-Do: 4. Vision API 요청 함수 정의 (request_image_analysis)
# 이 함수는 이미지 URL과 분석할 기능(features) 리스트를 받아 API에 요청을 보냅니다.
# **kwargs를 사용하여 'caption'이나 'smartCrops' 같은 특정 기능에 대한 추가 옵션(성 중립성, 종횡비 등)을 처리합니다.
# API 요청 실패 시 None을 반환하여 오류를 방지합니다.
def request_image_analysis(image_url, features=['objects'], **kwargs):
    endpoint = f"{BASE_ENDPOINT}"
    params = {
        "api-version": "2024-02-01",
        "features": ",".join(features)
    }
    headers = {"Ocp-Apim-Subscription-Key": API_KEY}
    body = {"url": image_url}

    if kwargs is not None:
        if ("caption" in features or "denseCaptions" in features) and "gender_neutral_caption" in kwargs:
            params['gender-neutral-caption'] = kwargs['gender_neutral_caption']
        if "smartCrops" in features and "smartcrops_aspect_ratios" in kwargs:
            params['smartcrops-aspect-ratios'] = kwargs['smartcrops_aspect_ratios']

    response = requests.post(endpoint, params=params, headers=headers, json=body)

    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}" )
        return None
    
    return response.json()

In [ ]:
# To-Do: 5. 분석 결과 시각화 함수 정의 (draw_image)
# API 응답(data)을 바탕으로 원본 이미지에 바운딩 박스와 텍스트(객체 이름, 캡션 등)를 그립니다.
# 'tags'나 'caption'처럼 이미지 전체에 대한 정보는 건너뛰고, 위치 정보가 있는 'objects', 'denseCaptions' 등을 처리합니다.
def draw_image(image_url, features, data):
    if data is None:
        return None
    image_response = requests.get(image_url)
    image = Image.open(BytesIO(image_response.content))
    draw = ImageDraw.Draw(image)
    font = get_font()

    for feature in features:
        if feature in ["tags", "caption"]:
            continue
        feature_key = f'{feature}Result'
        if feature_key in data and 'values' in data[feature_key]:
            result_object = data[feature_key]
            block_list = result_object['values']
            color = random_color()

            for block in block_list:
                if 'boundingBox' in block:
                    bounding_box = block['boundingBox']
                    x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
                    formatted_text = None

                    if feature == "objects" and 'tags' in block and block['tags']:
                        tag = block['tags'][0]
                        name = tag['name']
                        confidence = tag['confidence']
                        formatted_text = f"{name} ({confidence:.2f}%)"
                    elif feature == "denseCaptions":
                        formatted_text = block.get("text")

                    draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=3)
                    if formatted_text:
                        draw.text((x, y - 20), formatted_text, fill=color, font=font)
    return image

In [ ]:
# To-Do: 6. 텍스트 결과 생성 함수 정의 (get_result_text)
# API 응답에서 'caption'과 'tags' 정보를 추출하여 사람이 읽기 좋은 형태의 문자열로 만듭니다.
def get_result_text(features, response_data):
    if response_data is None:
        return "분석 결과를 가져오지 못했습니다."
    result_text_list = []
    if "caption" in features and 'captionResult' in response_data:
        result_text_list.append("[Caption]")
        result_text_list.append(response_data['captionResult']['text'])
        confidence = response_data['captionResult']['confidence'] * 100
        result_text_list.append(f"{confidence:.2f}%" )

    if "tags" in features and 'tagsResult' in response_data:
        if result_text_list:
            result_text_list.append("
")
        result_text_list.append("[Tags]")
        tag_list = response_data['tagsResult']['values']
        for tag in tag_list:
            name = tag['name']
            confidence = tag['confidence'] * 100
            result_text_list.append(f"{name} ({confidence:.2f}%)" )
            
    return "
".join(result_text_list)

### 여기까지가 함수화 과정입니다. 아래부터는 Gradio UI를 만드는 과정입니다.

In [ ]:
# To-Do: 7. Gradio 인터페이스 로직 함수 정의 (click_send)
# 이 함수는 Gradio UI에서 '전송' 버튼을 클릭했을 때 실행됩니다.
# 사용자가 입력한 이미지 URL과 선택한 옵션들을 받아 다음 작업들을 수행합니다.
# 1. request_image_analysis()를 호출하여 API에 분석을 요청합니다.
# 2. draw_image()를 호출하여 결과 이미지를 생성합니다.
# 3. get_result_text()를 호출하여 결과 텍스트를 생성합니다.
# 4. 생성된 이미지와 텍스트를 반환하여 UI에 표시합니다.
def click_send(image_url, features, is_gender_neutral, ratio):
    option = {}
    if "caption" in features or "denseCaptions" in features:
        option['gender_neutral_caption'] = is_gender_neutral
    if "smartCrops" in features:
        option['smartcrops_aspect_ratios'] = ratio

    response_data = request_image_analysis(image_url, features, **option)
    image = draw_image(image_url, features, response_data)
    result_text = get_result_text(features, response_data)

    return image, result_text

In [ ]:
# To-Do: 8. Gradio 인터페이스 동적 변경 함수 정의 (change_features)
# 사용자가 'Features' 체크박스에서 선택을 변경할 때마다 실행됩니다.
# 'caption'이나 'smartCrops'가 선택되었는지에 따라 관련 옵션(성 중립성 라디오 버튼, 종횡비 텍스트박스)을 보여주거나 숨깁니다.
def change_features(features):
    selected_caption = "caption" in features or "denseCaptions" in features
    selected_smart_crops = "smartCrops" in features
    return gr.update(visible=selected_caption), gr.update(visible=selected_smart_crops)

In [ ]:
# To-Do: 9. Gradio UI 구성 및 실행
# gr.Blocks()를 사용하여 전체 UI 레이아웃을 구성합니다.
# - CheckboxGroup: 분석할 기능을 다중 선택
# - Radio: 성 중립성 옵션 선택
# - Textbox: 이미지 URL 및 smartCrops 종횡비 입력
# - Button: 분석 시작
# - Image, TextArea: 결과 이미지와 텍스트 출력
# .click()과 .change()를 사용하여 각 UI 요소의 이벤트를 위에서 정의한 함수들과 연결합니다.
# demo.launch()를 호출하여 웹 앱을 실행합니다.

# with gr.Blocks() as demo:
#     FEATURES = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]
#
#     gr.Markdown("# Azure Vision API 이미지 분석기")
#
#     with gr.Row():
#         with gr.Column():
#             image_url_textbox = gr.Textbox(label="이미지 URL", value=IMAGE_URL)
#             features_checkbox = gr.CheckboxGroup(label="Features", choices=FEATURES, value=["objects", "caption"])
#             with gr.Box(visible=True) as caption_box:
#                 gender_radio = gr.Radio(label="성 중립성", choices=[("중립", True), ("구분", False)], value=True, interactive=True)
#             with gr.Box(visible=False) as smartcrops_box:
#                 ratio_textbox = gr.Textbox(label="smartCrops 종횡비", placeholder="예: 0.75,1.2", interactive=True)
#             send_button = gr.Button("분석 실행", variant="primary")
#
#         with gr.Column():
#             output_image = gr.Image(label="결과 이미지", type="pil")
#             output_textbox = gr.TextArea(label="결과 텍스트", lines=10)
#
#     # 이벤트 연결
#     features_checkbox.change(change_features, inputs=[features_checkbox], outputs=[caption_box, smartcrops_box])
#     send_button.click(click_send, 
#                       inputs=[image_url_textbox, features_checkbox, gender_radio, ratio_textbox], 
#                       outputs=[output_image, output_textbox])
#
# demo.launch()